# Identificar fraude no email da Enron 

### Primeira Parte: Fundamentação Teórica e Explicação do Código

O objetivo desse projeto é determinar se um funcionário é ou não um funcionário de interesse (POI). Um funcionário de interesse é um funcionário que participou do escândalo da empresa Enron. 

Para isso iremos usar diversos métodos que foram aprendidos no módulo de *Machine Learning*, mais especificamente usaremos o conjunto de métodos **sklearn**, que contém inúmeras funcionalidades para esse aprendizado :)

Meu código final está no arquivo *poi_id.py* que deve ser executado primeiro para exportar os conjuntos de dados e em seguida deve-se executar o *tester.py*.

Machine Learning foi a solução ideal para a resolução desse problema pois para sabermos se um funcionário está ou não envolvido no escândalo, nós precisamos que o nosso código saiba o que é isso e como classificar isso.
Existem diversos tipos de ML tais como recomendação (que Netflix e Amazon usam e abusam), detecção de anomalias, prevenção de fraude, agrupamento e diversos outros.

Para a resolução desse problema eu testei **4 algoritmos** que aprendi ao longo do módulo:
- **Decision Tree:** Esse algoritmo pode ser comparado ao *CASE WHEN... THEN.. ELSE... END". Basicamente nós aplicamos certas regras nos dados e dependendo de cada valor, um decisão é tomada. Esse algoritmo é bastante usado em pesquisas operacionais.
- **Random Forest:** É um *ensemble learning* supervisionado, que é comumente usado em problemas de classificação, detecção de anomalias e redes neurais.
- **SDG:** Esse método é usado em otimização e tem como objetivo encontrar o mínimo local de uma função.
- **NaiveBayes:** É bastante usado em *text learning* 

Além dos 4 métodos apresentados acima, existem alguns cuidados, ou melhor, tratativas que devem ser aplicadas no dataset antes de treinar o algoritmo.

Um ponto bastante importante é a **remoção de outliers**. Classificamos um valor como outlier quando ele está muito discrepante do restante dos valores. Existem algumas técnicas para a detecção/exclusão, uma bastante conhecida e que aprendi aqui no Nanodegree é o [Interquartile Range (IQR)](https://en.wikipedia.org/wiki/Interquartile_range).

Nesse projeto tive que remover um valor do conjunto de dados. 
   - `data_dict.pop('TOTAL')`: *TOTAL* não é uma pessoa e isso estava poluindo a análise.
   
Algumas métricas iniciais sobre o nosso conjunto de dados:
   - **Total de registros:**  145
   - **Total de poi:**  18
   - **Total de não-poi:**  127
   - **Total de atributos:** 23
   
Outro ponto de limpeza dos dados que precisei fazer foi realizar um replace em todos os registros cujos valores eram **NaN**. Isso poderia afetar bastante as análises, então decidi apenas substituir por **0**:
   - `final = df[cols].copy().applymap(lambda x: 0  if x == 'NaN' else x)`
   
Agora sobre as **features** que foram analizadas aqui, utilizei o algoritmo [SelectPercentile](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectPercentile.html) e [FeatureSelection](http://scikit-learn.org/stable/modules/feature_selection.html) para saber quais features mais eram relevantes para a descoberta da nossa pergunta.
O primeiro foi para pegar um percentual do conjunto de dados e o segundo foi utilizado para a seleção das features de fato :)

Durante o módulo, nós aprendemos a criar novas features que podem nos ajudar durante a análise... sabendo disso, decidi criar uma funçao chamada *computeFractionPoi*, que retorna o percentual de mensagens que a pessoa enviou para um *POI* e que recebeu de um *POI*.

**Por que achei isso útil?**
   - Bom, com esses dois valores nós conseguimos saber se a pessoa tinha muito contato com um POI, assim podemos supor que como ela tem muito contato com ele, ela também pode ser um
   
Infelizmente, essas duas variáveis criadas não tiveram o resultado que eu esperava.
Mais pra frente no relatório você vai ver que ela não foi classificada com um das variáveis (features) mais importantes para o nosso modelo :(

**Ajustes de escala:**
   - Aqui utilizei o método [MinMaxScaler()](http://scikit-learn.org/stable/modules/preprocessing.html) para ajuste nas escalas dos dados. A motivação para usar esta escala inclui robustez a desvios padrões muito pequenos de recursos e preservando zero entradas em dados escassos.

Tendo nossas features selecionadas e armazenadas em *features_list*, executei o seguinte código para fazer a divisão em **labels** e **features** para que possamos começar a testar os algoritmos de ML!

```
data = featureFormat(my_dataset, features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)
```

Esse conjunto *featureFormat* foi provido pela Udacity e pode ser encontrado [aqui](https://github.com/udacity/ud120-projects/blob/master/tools/feature_format.py).

Por fim, utilizei o método de [cross_validation](http://scikit-learn.org/stable/modules/cross_validation.html) para realizarmos a divisão de nosso dataset em dois, um para teste e outro para treinamento. 
Realizar essa divisão é essencial, pois ele nos ajuda a evitarmos bastante a ocorrência de *overfitting*, porém alguns cuidados devem ser tomados, como por exemplo, a divisão precisa ser randômica, para que não treinamos nosso algoritmo e deixe ele com um viés mais forte. 

Para isso, é usado o [StratifiedShuffleSplit](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html), que basicamente garante que teremos uma quantidade proporcional de POI vs non-POI. 
Isso é usado porque pode acontecer de ao usarmos um método comum para split dos dados, pode acontecer de não termos rótulos de POI no conjunto de teste, o que não tornaria o modelo bom o suficiente.

Para isso, dentro desse método, existe o [test_train_split](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn.model_selection.train_test_split) que faz essa divisão e já leva em conta esse cuidado.

Aqui eu fiz a divisão clássica de 30% para teste e 70% para treino.

Na segunda parte entrarei um pouco mais detalhado no código e em alguns testes e tunings que fiz com alguns parâmetros para que fosse possível obter a melhor resposta :)


### Antes de irmos para o código de fato, vou falar um pouquinho sobre o **porque devemos utilizar tuning**.

Quando se usa um algoritmo de machine learning, basicamente queremos prever algo utilizando nossos dados, nossas features como fonte da "inteligência". Ao usarmos os valores defaults para os parâmetros desses algoritmos, pode ser que eles não seja os ideias para o nosso caso e nos retornem um resultado diferente do que estamos esperando.
Ou também quando precisamos de performance dificilmente os valores padrão dos parâmetros nos trarão a performance que queremos.

Aqui no relatório, por exemplo, eu fiz testes com o parâmetro do SelectPercentile para que pudesse escolher realmente as melhores features para esse problema. Quando você precisa que seu algoritmo seja usado em sua máxima "potência", você precisa entender como ele funciona, para que serve cada parâmetro e, com certeza, alterar esses valores até chegar em um ponto que te deixe satisfeito :)

### Segunda Parte: Código executado

In [1]:
import sys
import os
sys.path.append("../ud120-projects/")

In [4]:
## DecisionTreeeClassifier
run tester.py

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
	Accuracy: 0.76691	Precision: 0.35142	Recall: 0.33350	F1: 0.34223	F2: 0.33694
	Total predictions: 11000	True positives:  667	False positives: 1231	False negatives: 1333	True negatives: 7769



In [7]:
## DecisionTreeeClassifier
run tester.py

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)
	Accuracy: 0.82473	Precision: 0.54380	Recall: 0.22350	F1: 0.31680	F2: 0.25334
	Total predictions: 11000	True positives:  447	False positives:  375	False negatives: 1553	True negatives: 8625



In [11]:
## NaiveBayesClassifier
run tester.py

GaussianNB(priors=None)
	Accuracy: 0.83255	Precision: 0.56280	Recall: 0.35400	F1: 0.43462	F2: 0.38237
	Total predictions: 11000	True positives:  708	False positives:  550	False negatives: 1292	True negatives: 8450



In [11]:
## SDGClassifier with GridSearchDV
run tester.py

GridSearchCV(cv=10, error_score='raise'
	estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1
	eta0=0.0, fit_intercept=True, l1_ratio=0.15
	learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1
	penalty='l2', power_t=0.5, random_state=42, shuffle=True, verbose=0
	warm_start=False)
	fit_params={}, iid=True, n_jobs=1
	param_grid={'loss': ['hinge', 'log', 'squared_hinge'], 'n_iter': [30, 35], 'alpha': [0.01, 0.0001, 1e-06]}
	pre_dispatch='2*n_jobs', refit=True, scoring='f1', verbose=0)
	Accuracy: 0.60445 Precision: 0.16036  Recall: 0.27750 F1: 0.20326 F2: 0.24213
	Total predictions: 11000  True positives:  555  False positives: 2906 False negatives: 1445 True negatives: 6094



Como pode-se perceber, aqui não precisei tunar o modelo para conseguir o mínimo requerido para o projeto (0.3 para ambas as métricas).

Quando calculamos a métrica de precisão, queremos saber qual é a razão entre os eventos que previmos corretamente com todos os outros que eram corretos, ou seja, se em nosso conjunto previmos que temos 100 POIs, mas na verdade apenas 30 realmente são, significa que tivemos uma precisão de 30%.
Já recall é quando queremos saber a taxa de verdadeiros positivos no problema, ou seja, se no conjunto todo temos 300 POIs, significa que tivemos 33% de recall (100/300).

Um tratamento que fiz foi selecionar o melhor *percentile* para o conjunto de dados. O que esse método faz é selecionar as **features de acordo com o seu score**.

Seguem abaixo alguns testes que fiz até chegar no melhor modelo :)

Eu usei o NaiveBayes como teste para achar o melhor Percentile :)

Meu conjunto de features final ficou sendo o seguinte:

### Terceira Parte: Possíveis problemas com o conjunto de dados

Aqui irei pontuar em tópicos alguns problemas que encontrei nesse conjunto de dados:
   - Inconsistência nos e-mails
   - Diferentes conjuntos de dados podem introduzir diferentes bias e erros
   - POI pode não estar no conjunto de dados